# Evolver Loop 12 Analysis

## Key Findings from Research

### 1. Two Novel Techniques Discovered (NOT TRIED YET!)

**A. "Rebuild from Corners" (chistyakov kernel)**
- For each large N layout, check all 4 corners
- Sort trees by distance from corner
- Reconstruct smaller layouts by taking closest trees
- If subset beats existing solution for smaller N, use it
- **Pure Python, no optimizer needed!**

**B. "BackPacking" (crodoc kernel)**
- Start from N=200, iterate backward to N=1
- Track best configuration at each step
- When smaller N has worse score, copy from best larger config and drop extras
- Propagates good packing patterns from large N to small N
- **Pure Python, no optimizer needed!**

### 2. Current State
- All 115 local snapshots converge to same local optimum (70.615107)
- Ensemble found only 1.76e-7 valid improvement (numerical noise)
- The "improvements" from exp_002/004/006 came from INVALID snapshot 21145966992
- Python SA, fractional translation, random restart all found ZERO improvements

### 3. Path Forward
The two novel techniques above have NOT been tried yet. They don't require:
- Running any binary optimizer
- C++ compilation
- External data sources

They work by EXTRACTING better solutions from existing layouts.

In [ ]:
# Let's verify the baseline and understand the per-N score distribution
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 25
scale_factor = Decimal('1e18')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        
        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor)
        )

def get_side_length(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1]) / float(scale_factor)

# Load baseline
baseline_path = "/home/nonroot/snapshots/santa-2025/21337353543/submission/submission.csv"
df = pd.read_csv(baseline_path)
df['x'] = df['x'].str.strip('s')
df['y'] = df['y'].str.strip('s')
df['deg'] = df['deg'].str.strip('s')

print(f"Loaded baseline with {len(df)} rows")

In [ ]:
# Calculate per-N scores for baseline
baseline_scores = {}
for n in range(1, 201):
    group_data = df[df['id'].str.startswith(f'{n:03d}_')]
    trees = [ChristmasTree(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]
    side = get_side_length(trees)
    score = (side ** 2) / n
    baseline_scores[n] = score

total_score = sum(baseline_scores.values())
print(f"Baseline total score: {total_score:.6f}")
print(f"\nTop 10 highest per-N scores (most room for improvement):")
sorted_scores = sorted(baseline_scores.items(), key=lambda x: -x[1])
for n, score in sorted_scores[:10]:
    print(f"  N={n}: {score:.6f}")

In [ ]:
# Test the "rebuild from corners" technique on a sample large N
# This extracts smaller N solutions from larger N layouts

def rebuild_from_corners(large_n_trees, target_n):
    """Extract target_n trees from large_n layout using corner-based selection."""
    if len(large_n_trees) < target_n:
        return None
    
    all_polygons = [t.polygon for t in large_n_trees]
    bounds = unary_union(all_polygons).bounds
    
    best_subset = None
    best_side = float('inf')
    
    # Try all 4 corners
    for corner_x, corner_y in [(bounds[0], bounds[1]), (bounds[0], bounds[3]), 
                                (bounds[2], bounds[1]), (bounds[2], bounds[3])]:
        # Calculate distance from corner for each tree
        distances = []
        for i, tree in enumerate(large_n_trees):
            tree_bounds = tree.polygon.bounds
            dist = max(
                abs(tree_bounds[0] - corner_x),
                abs(tree_bounds[2] - corner_x),
                abs(tree_bounds[1] - corner_y),
                abs(tree_bounds[3] - corner_y)
            )
            distances.append((dist, i))
        
        # Sort by distance and take closest target_n trees
        distances.sort()
        subset_indices = [idx for _, idx in distances[:target_n]]
        subset = [large_n_trees[i] for i in subset_indices]
        
        # Calculate side length
        side = get_side_length(subset)
        if side < best_side:
            best_side = side
            best_subset = subset
    
    return best_subset, best_side

# Test on N=111 -> extract N=10, N=20, N=30
test_large_n = 111
group_data = df[df['id'].str.startswith(f'{test_large_n:03d}_')]
large_trees = [ChristmasTree(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]

print(f"Testing 'rebuild from corners' on N={test_large_n}:")
for target_n in [10, 20, 30, 50, 100]:
    subset, new_side = rebuild_from_corners(large_trees, target_n)
    new_score = (new_side ** 2) / target_n
    baseline_score = baseline_scores[target_n]
    improvement = baseline_score - new_score
    status = "✅ BETTER" if improvement > 1e-8 else "❌ same/worse"
    print(f"  N={target_n}: baseline={baseline_score:.6f}, extracted={new_score:.6f}, diff={improvement:.8f} {status}")

In [ ]:
# Test on ALL large N values to find improvements
print("Searching for improvements using 'rebuild from corners'...")
print("(Testing all large N from 50-200 to extract smaller N solutions)\n")

improvements = []

for large_n in range(50, 201):
    group_data = df[df['id'].str.startswith(f'{large_n:03d}_')]
    large_trees = [ChristmasTree(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]
    
    for target_n in range(1, large_n):
        subset, new_side = rebuild_from_corners(large_trees, target_n)
        new_score = (new_side ** 2) / target_n
        baseline_score = baseline_scores[target_n]
        improvement = baseline_score - new_score
        
        if improvement > 1e-8:  # Significant improvement
            improvements.append({
                'target_n': target_n,
                'source_n': large_n,
                'baseline_score': baseline_score,
                'new_score': new_score,
                'improvement': improvement
            })

if improvements:
    print(f"Found {len(improvements)} improvements!")
    improvements_df = pd.DataFrame(improvements)
    improvements_df = improvements_df.sort_values('improvement', ascending=False)
    print("\nTop 20 improvements:")
    print(improvements_df.head(20).to_string(index=False))
    print(f"\nTotal potential improvement: {improvements_df['improvement'].sum():.6f}")
else:
    print("No improvements found with 'rebuild from corners' technique.")
    print("The baseline layouts are already well-optimized for this approach.")